# VacationPy

In [88]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [89]:
va_city_data_df = pd.read_csv("../WeatherPy/city_data.csv")
va_city_data_df.head()

,Unnamed: 0,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Airai,-8.93,125.41,60.84,100,16,0.89,TL,1603827026
1,1,Cape Town,-33.93,18.42,66.00,82,20,4.70,ZA,1603826904
2,2,Vaini,-21.20,-175.20,69.80,94,75,5.82,TO,1603826444
3,3,George Town,5.41,100.34,78.80,88,20,5.77,MY,1603826490
4,4,Krasnokamensk,50.10,118.04,23.16,85,0,8.72,RU,1603827026


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [90]:
#configure gmaps with API key
gmaps.configure(api_key=g_key)

In [91]:
#check datatypes for lat and lng
locations = va_city_data_df[["Lat", "Lng"]].astype(float)
locations.dtypes

Lat    float64
Lng    float64
dtype: object

In [92]:
#check datatypes for humidity
humidity = va_city_data_df["Humidity"]
humidity.dtypes

dtype('int64')

In [93]:
#create heatmap layer for locations and humidity
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Defining parameters to find locations with ideal weather conditions: 
1. Temperatures between 72-78 degrees Fahrenheit
2. Humidity below 80%
3. Cloudiness less than 25%
4. Wind Speed below 5 mph

In [95]:
#create dataframe with only the columns we need (drop date column)
va_data_df = va_city_data_df.loc[:,["City Name","Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country"]]
va_data_df.head()

#narrow down criteria 
narrow_va_data_df = va_data_df.loc[(va_data_df["Max Temp"]>72) & (va_data_df["Max Temp"]<78) &
                                   (va_data_df["Humidity"]<80) & (va_data_df["Cloudiness"]<25) &
                                   (va_data_df["Wind Speed"]<5)]

narrow_va_data_df                                    

#drop any rows with null values 
cleaned_va_data = narrow_va_data_df.dropna(how="any")
cleaned_va_data

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country
117,Qurayyat,31.33,37.34,77.00,12,0,4.70,SA
170,Mersin Province,36.86,34.65,73.40,38,0,2.13,TR
197,Lompoc,34.64,-120.46,73.40,12,1,4.43,US
306,San Francisco,37.77,-122.42,77.00,20,20,3.36,US
325,Santa Barbara,34.73,-120.03,73.99,33,1,3.36,US
360,Wilmington,34.23,-77.94,75.99,77,10,1.01,US
419,Bilma,18.69,12.92,73.98,20,0,3.83,NE
431,Rancho Palos Verdes,33.74,-118.39,75.00,17,0,1.99,US
514,Wamba,8.93,8.60,75.25,52,0,2.51,NG
521,Gaurnadi,22.97,90.23,75.11,79,0,4.56,BD


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [96]:
#create a new dataframe for hotel info
hotel_df = cleaned_va_data

#add hotel name column 
hotel_df["Hotel Name"] = ""
hotel_df

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
117,Qurayyat,31.33,37.34,77.00,12,0,4.70,SA,
170,Mersin Province,36.86,34.65,73.40,38,0,2.13,TR,
197,Lompoc,34.64,-120.46,73.40,12,1,4.43,US,
306,San Francisco,37.77,-122.42,77.00,20,20,3.36,US,
325,Santa Barbara,34.73,-120.03,73.99,33,1,3.36,US,
360,Wilmington,34.23,-77.94,75.99,77,10,1.01,US,
419,Bilma,18.69,12.92,73.98,20,0,3.83,NE,
431,Rancho Palos Verdes,33.74,-118.39,75.00,17,0,1.99,US,
514,Wamba,8.93,8.60,75.25,52,0,2.51,NG,
521,Gaurnadi,22.97,90.23,75.11,79,0,4.56,BD,


In [97]:
#define radius and type 
hotels = "lodging"
radius = 5000

#set parameters
params = {
    "types": hotels,
    "radius": radius,
    "key": g_key
}

#iterate through hotel_df: 
for index, row in hotel_df.iterrows():
    
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #run request
    data = requests.get(base_url, params=params).json()

    results = data["results"]
    
    try: 
        hotel_name = data["results"][0]["name"]
        print(f"Retrieving Results for Index {index}: {hotel_name}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotel_name

    except (KeyError, IndexError): 
        print("Cannot find hotel for", row["City Name"])
        
    print("------------------")    

Retrieving Results for Index 117: Mozon Furnished Aprtments.
------------------
Retrieving Results for Index 170: Nostalgia Boutique Hotel.
------------------
Retrieving Results for Index 197: Holiday Inn Express Lompoc.
------------------
Retrieving Results for Index 306: The Clift Royal Sonesta Hotel.
------------------
Retrieving Results for Index 325: Upper Ballard Campground.
------------------
Retrieving Results for Index 360: Hotel Ballast Wilmington, Tapestry Collection by Hilton.
------------------
Retrieving Results for Index 419: Expeditions Ténére voyages.
------------------
Retrieving Results for Index 431: Terranea Resort.
------------------
Retrieving Results for Index 514: Roma Zee Guest House Annex.
------------------
Retrieving Results for Index 521: Govt Gournadi College Dormitory.
------------------
Retrieving Results for Index 555: Hotel Praia Morena.
------------------


In [98]:
#display in dataframe
hotel_df

#delete row with no hotel found
empty_row = hotel_df.loc[hotel_df["Hotel Name"] == ""]
index = empty_row.index
hotel_df = hotel_df.drop(index)

#display dataframe with only complete data
hotel_df

,City Name,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
117,Qurayyat,31.33,37.34,77.00,12,0,4.70,SA,Mozon Furnished Aprtments
170,Mersin Province,36.86,34.65,73.40,38,0,2.13,TR,Nostalgia Boutique Hotel
197,Lompoc,34.64,-120.46,73.40,12,1,4.43,US,Holiday Inn Express Lompoc
306,San Francisco,37.77,-122.42,77.00,20,20,3.36,US,The Clift Royal Sonesta Hotel
325,Santa Barbara,34.73,-120.03,73.99,33,1,3.36,US,Upper Ballard Campground
360,Wilmington,34.23,-77.94,75.99,77,10,1.01,US,"Hotel Ballast Wilmington, Tapestry Collection ..."
419,Bilma,18.69,12.92,73.98,20,0,3.83,NE,Expeditions Ténére voyages
431,Rancho Palos Verdes,33.74,-118.39,75.00,17,0,1.99,US,Terranea Resort
514,Wamba,8.93,8.60,75.25,52,0,2.51,NG,Roma Zee Guest House Annex
521,Gaurnadi,22.97,90.23,75.11,79,0,4.56,BD,Govt Gournadi College Dormitory


In [99]:
# NOTE: Do not change any of the code in this cell
##changed "City" variable to "City Name" to match my df column name

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [100]:
# Add marker layer
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [102]:
#create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Google heatmap with hotel markers observation: 
As shown in the map above, after narrowing down possible vacation locales, most are in areas with moderate humidity. On a personal note, I was pleased to see the city of Wilmington, NC included. It's a place I know well from my four years attending UNC-Wilmington. I can recommend Wilmington as a pleasant place to visit, which boosts my confidence that the other locations would have great weather for traveling as well. 